# Introduction to the Julia programming language

** Andy Ferris, 16 November 2017 (revised 16 April 2018) **

This document is a short introduction to Julia, and in particular aims to:

 * Get you to install Julia
 * Motivate why I beleive Julia is a useful tool
 * Explain the basics of *how* the language and compiler work
 * Teach some basic usage

This document is called a "Jupyter notebook", and allows one to intersperse markdown text, HTML, running code, results and graphs. I will share a link with you to view and run the examples below.

Here is an example of the live programming environment (use shift-enter to evaluate a code block):

In [1]:
1 + 1

2

## Installing Julia

The easiest way to install Julia is to visit https://julialang.org/downloads and click the download relevant for your OS. For 64-bit linux and Julia 0.6.2 you will want to extract files from the .tar.gz and rename the directory like so:
```
cd ~
tar -xzf Downloads/julia-0.6.2-linux-x86_64.tar.gz 
mv julia-d386e40c17 julia-0.6.2
```

You may also want to add `julia` to your path

```
export PATH=$PATH:~/julia-0.6.2/bin
echo 'export PATH=$PATH:~/julia-0.6.2/bin' >>~/.bash_profile
```

Then you can begin an interactive session by typing:
```
julia
```

The interactive prompt you see is called the REPL for "Read-Evaluate-Print-Loop". It looks like:

```
   _       _ _(_)_     |  A fresh approach to technical computing
  (_)     | (_) (_)    |  Documentation: https://docs.julialang.org
   _ _   _| |_  __ _   |  Type "?help" for help.
  | | | | | | |/ _` |  |
  | | |_| | | | (_| |  |  Version 0.6.2 (2017-12-13 18:08 UTC)
 _/ |\__'_|_|_|\__'_|  |  
|__/                   |  x86_64-linux-gnu

julia> 1 + 1
2

julia> 
```

Julia's REPL has some useful features like history (↑ and ↓), autocompletion (`tab`), help mode (`?`) and shell mode (`;`). You can define variables and functions that can be used later on, like:

In [2]:
x = 3

2*x

6

In [3]:
f(x) = x + 1

f(10)

11

In [12]:
# Line cmments begin with hash
#= There are also comment blocks (which can #= nest =# correctly!) =#

# We can also use a "long" function form:
function f(x)
    return x + 1
end

f(10)

11

## Why Julia?

Simple answer: **programmer productivity**. I can get more stuff done.

My history is in numerical computing, where I would typically do the following:

 1. Prototype an algorithm in MATLAB
 2. Rewrite my code in C++ and deploy to HPC cluster
 
The reasons I did this were

 1. High-level languages like MATLAB (python, R, etc) let one explore and experiment rapidly, but can run slow.
 2. Low-level languages like C++ tend to take longer to develop, but run fast.

This is sometimes called the "two language problem" and is something the Julia developers set out to eliminate. With Julia, I find:

 1. I can write code in *much* less time than C++, and quicker than MATLAB.
 2. My code runs *much* faster than MATLAB, and typically similar to C++ (function inlining and SIMD are better in Julia, while is garbage collection is slower).
 
I would argue this provides a "best of both worlds" experience for programmers who need to develop novel algorithms and bring them into production environments with minimal effort.

## What makes a language "productive"

 * Readable code
 * Generic, reusable code
 * Ecosystem of **relevant** libraries and packages
 * Easy to understand and remember APIs (concistent semantics across the standard library and packages)
 * Sound software engineering foundations (e.g. unit testing framework, no `null` objects)
 * Dynamic enough for prototyping and polymorphism, with the safety of static typing when required
 * Able to use a variety of programming styles (functional programming, object oriented, procedural, etc)
 * Can interface with other programming languages (a good FFI)

Julia tends to approach these with a powerful type system, a well curated standard library (written entirely in Julia itself) and an interesting JIT compiler. More on that later.

## What makes a language "fast"

Take MATLAB as an example - MATLAB *is* extremely fast for certain programming styles (involving vectorized operations - adding two arrays, for example). However, as soon as you use recursive function calls, `for` loops or `if` statements, everything is slow!

The point is this: a language should be fast (as fast as possible) for all the language constructs it provides, otherwise programmers will fall into performance traps and/or need avoid certain language features. The [Julia home page](https://julialang.org) shows table comparing the speed of certain benchmarks - it is important to note that each benchmark is designed to test the speed of a language feature, such as performing lots of recursive function calls, or iterating over an array with a `for` loop, etc.

Here's an example of the benchmark code (this tests `for` and `if` and basic math):

In [4]:
function mandel(z)
    c = z
    maxiter = 80
    for n in 1:maxiter
        if abs2(z) > 4
            return n-1
        end
        z = z^2 + c
    end
    return maxiter
end

mandel (generic function with 1 method)

Now let's make an image of the fractal Mandlebrot set!

First we use the package manager to get a plotting library for us.

In [6]:
using Pkg
Pkg.add("Plots")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %              ]  62.6 %Fetching: [==========================>              ]  63.1 %>  ]  94.3 % Resolving package versions...
 Installed Widgets ─ v0.5.0
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
  [cc8bc4a8] ↑ Widgets v0.4.4 ⇒ v0.5.0


The first time you load the package it will do some compilation, etc.

In [7]:
using Plots

┌ Info: Recompiling stale cache file /home/olszewskip/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


The *Plots* library supports multiple backends to do the drawing. We'll choose *plotly* which is a Javascript library

In [8]:
 plotly()

Plots.PlotlyBackend()

Now we make the fractal and plot it.

In [10]:
# The domain of the image
xs = -2.0:0.01:0.5 # A range of numbers between -2.0 and 0.5, with spacing 0.01
ys = -1.25:0.01:1.25

# Calculate how many iterations before diverging
iters = [mandel(x + y*im) for x in xs, y in ys] # an "array comprehension"

# Make the plot
heatmap(xs, ys, iters, aspect_ratio = 1.0)

---------
## Julia types and multiple dispatch

Each object in Julia has a **concrete type**.

In [11]:
0

0

In [12]:
0.0

0.0

In [13]:
typeof(1)

Int64

In [14]:
typeof(3.14)

Float64

In [15]:
typeof(pi)

Irrational{:π}

In [16]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [17]:
typeof([1,2,3])

Array{Int64,1}

In [18]:
1:3

1:3

In [19]:
typeof(1:3)

UnitRange{Int64}

### Concrete types

You can think of a type as a collection objects. If `x` is of type `T` then we write `x::T`. The `::` operator makes an assertion, and is used in function signatures.

In [20]:
1 :: Int64

1

In [21]:
1 :: String

TypeError: TypeError: in typeassert, expected String, got Int64

In [22]:
function add(x::Int64, y::Int64)
    return x + y
end

add(3, 4)

7

In [23]:
# Note that `struct`s by default are immutable - the fields `name`, `age` and `is_alive` 
# cannot be modified after construction. There is also `mutable struct`.

struct Person
    name::String
    age::Int32
    is_alive::Bool
end

Person("Andy", 34, true)

Person("Andy", 34, true)

### Abstract types

Types can also be abstact - they represent combinations of concrete types. Julia has a single-inheretence type tree. If type `A` is a subtype of type `B`, then we write `A <: B` (which is a predicate that returns a `Bool` - `true` or `false`). There is a common supertype called `Any`.

Some important type trees include:

```
Int64 <: Signed <: Integer <: Real <: Number <: Any
Array{T, N} <: DenseArray{T,N} <: AbstractArray{T,N} <: Any
String <: AbstractString <: Any
```

There is also a `Union` type, so that `Union{Int64, Float64}` is an abstract type which contains objects which are `Int64` or `Float64`.

In [24]:
abstract type MySuper; end

struct MyType{T} <: MySuper
    a::T
    b::T
end

x = MyType(1, 1)
println(typeof(x))
println(supertype(typeof(x)))

MyType{Int64}
MySuper


### Multiple dispatch

Julia functions can have many methods attached to them, accepting different types (types do not typically have member methods). The most specific methodn which matches your (concrete) inputs will be chosen.

For example functions like `+` have many methods depending on what is being added.

In [25]:
function func(x::Any, y::Any)
    return x
end

function func(x::Float64, y::Float64)
    return y
end

println(func(1, 2))
println(func(1.0, 2.0))

1
2.0


In [26]:
methods(func)

# 2 methods for generic function "func":
[1] func(x::Float64, y::Float64) in Main at In[25]:6
[2] func(x, y) in Main at In[25]:2

### Compilation

When a function is called for the first time with certain types, the Julia compiler will  *specialize*  the function based on the concrete types and compile the result to native code.

The code is *fast* because it is based on the concrete types (made up of C-like structs) and uses LLVM to optimize.

There is an inference engine which propagates type information through the function and gives the return type. Sometimes the inference engine can correctly infer all types, and sometimes it is ambiguous. When it succeeds, all inner function calls can also be specialized at compile-time, and the resulting code is comparable to a statically compiled language.

If inference fails, the code still works but parts will run at slower speeds (like an interpretted language) because function calls with unknown types have to undergo *dynamic multiple dispatch*. The general task of selecting the correct method to dispatch to at run time is difficult, so this code is slower than using vtables.

In [27]:
@code_native func(1, 2)

	.text
; ┌ @ In[25]:2 within `func'
	movq	%rdi, %rax
	retq
	nopw	%cs:(%rax,%rax)
; └


### Metaprogramming

Julia has a powerful metaprogramming interface with macros, generated functions and more. Julia is like LISP in that it is a dynamic language and that code is represented in the language. Users can create macros that make arbitrary code transformations - they start with a `@` to indicate that they are macros to make their use more obvious when reading the code.

In [28]:
macro add(x, y)
    return :($x + $y)
end

@add 2 3      # or @add(2, 3) 

5

## Arrays

Julia arrays are multi-dimensional. 1D arrays are called vectors, 2D arrays are matrices. MATLAB style syntax is used for constructing them, like:

```julia
vector = [1, 2, 3]

matrix = [1 2; 
          3 4]
```

`Vector{T}` is an alias for an `Array{T, 1}` and `Matrix{T}` is an alias for `Array{T, 2}`


### Vectors as deques

`Vector{T}` is used both like a C++ `std::vector<T>`, as well as a linear algebra vector. We can

 * `push!(vector, element)` to add an element to the end of `vector`
 
 * `element = pop!(vector)` to remove an element from the end of vector
 
 * `insert!`, `append!`, etc.
 
Most functions in Julia are "pure" - they do not mutate their inputs. Ones which do tend to end in `!` to make it obvious that something will change.

In [29]:
list = Vector{Int64}()

0-element Array{Int64,1}

In [30]:
push!(list, 42)
list

1-element Array{Int64,1}:
 42

### Ranges

A unit range can be constructed as `1:3`, which will be equivalent to `[1,2,3]` however it is stored as a `UnitRange` which is a "lazy" container only storing the first and last element of the range. There are also step ranges using syntax `start:step:stop`.

### Comprehensions

To save time making arrays, comprehensions let us fill an array depending on some other inputs. Earlier, we saw

```julia
iters = [mandel(x + y*im) for x in xs, y in ys]
```

This array comprehnension is equivalent to something like:

```julia
iters = Array{Int}(length(xs), length(ys)) # Allocate the memory for a 2D array

for i in 1:length(xs)
    for j in 1:length(ys)
        iters[i, j] = mandel(xs[i] + im * ys[i])
    end
end
```

### Linear algebra

A full linear algebra system is provided, similar to MATLAB. Vector addition, matrix multiplication, eigenvalue decomposition, and so-on are supported.

In [43]:
using LinearAlgebra

In [44]:
matrix = [1 2; 3 4]
vector = [10, 20]

matrix * vector

2-element Array{Int64,1}:
  50
 110

In [47]:
?eigen

search: eigen eigen! Eigen GeneralizedEigen weighted_color_mean eigvecs



```
eigen(A; permute::Bool=true, scale::Bool=true) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an `Eigen` factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

For general nonsymmetric matrices it is possible to specify how the matrix is balanced before the eigenvector calculation. The option `permute=true` permutes the matrix to become closer to upper triangular, and `scale=true` scales the matrix by its diagonal elements to make rows and columns more equal in norm. The default is `true` for both options.

# Examples

```jldoctest
julia> F = eigen([1.0 0.0 0.0; 0.0 3.0 0.0; 0.0 0.0 18.0])
Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
  1.0
  3.0
 18.0
eigenvectors:
3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

julia> F.values
3-element Array{Float64,1}:
  1.0
  3.0
 18.0

julia> F.vectors
3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

julia> vals, vecs = F; # destructuring via iteration

julia> vals == F.values && vecs == F.vectors
true
```

---

```
eigen(A, B) -> GeneralizedEigen
```

Computes the generalized eigenvalue decomposition of `A` and `B`, returning a `GeneralizedEigen` factorization object `F` which contains the generalized eigenvalues in `F.values` and the generalized eigenvectors in the columns of the matrix `F.vectors`. (The `k`th generalized eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

# Examples

```jldoctest
julia> A = [1 0; 0 -1]
2×2 Array{Int64,2}:
 1   0
 0  -1

julia> B = [0 1; 1 0]
2×2 Array{Int64,2}:
 0  1
 1  0

julia> F = eigen(A, B);

julia> F.values
2-element Array{Complex{Float64},1}:
 0.0 + 1.0im
 0.0 - 1.0im

julia> F.vectors
2×2 Array{Complex{Float64},2}:
  0.0-1.0im   0.0+1.0im
 -1.0-0.0im  -1.0+0.0im

julia> vals, vecs = F; # destructuring via iteration

julia> vals == F.values && vecs == F.vectors
true
```

---

```
eigen(A::Union{SymTridiagonal, Hermitian, Symmetric}, irange::UnitRange) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an `Eigen` factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

The `UnitRange` `irange` specifies indices of the sorted eigenvalues to search for.

!!! note
    If `irange` is not `1:n`, where `n` is the dimension of `A`, then the returned factorization will be a *truncated* factorization.


---

```
eigen(A::Union{SymTridiagonal, Hermitian, Symmetric}, vl::Real, vu::Real) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an `Eigen` factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

`vl` is the lower bound of the window of eigenvalues to search for, and `vu` is the upper bound.

!!! note
    If [`vl`, `vu`] does not contain all eigenvalues of `A`, then the returned factorization will be a *truncated* factorization.



In [49]:
vals, vecs = eigen(matrix)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
2-element Array{Float64,1}:
 -0.3722813232690143
  5.372281323269014 
eigenvectors:
2×2 Array{Float64,2}:
 -0.824565  -0.415974
  0.565767  -0.909377

In [53]:
?tr

search: tr try triu tril trunc trues triu! tril! trylock tryparse truncate



```
tr(M)
```

Matrix trace. Sums the diagonal elements of `M`.

# Examples

```jldoctest
julia> A = [1 2; 3 4]
2×2 Array{Int64,2}:
 1  2
 3  4

julia> tr(A)
5
```


In [52]:
sum(vals) == tr(matrix)

true

### Higher order programming

A range of higher-order functions exist like

 * `map(f, c)` - creates a new container where each element goes through function `f` 
 * `map!(f, c)` - like `map` but mutates the input
 * `reduce(op, c)` - iterates through the values and applies `op` sequentially. E.g. if `op == +` then this is `sum`.
 * `broadcast` - a bit like `map` but expands out scalars and orthogonal dimensions of multidimensional arrays
 * ...

Functions like `f` are Julia objects with their own type. The operation is specialized on the function and inlining removes

In [54]:
vector = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [55]:
# short for `transpose([1, 2, 3])`
rowvector = [1,2,3]'

1×3 Adjoint{Int64,Array{Int64,1}}:
 1  2  3

In [56]:
?Adjoint

search: Adjoint adjoint adjoint!



No documentation found.

`LinearAlgebra.Adjoint` is of type `UnionAll`.

# Summary

```
struct UnionAll <: Type{T}
```

# Fields

```
var  :: TypeVar
body :: Any
```

# Supertype Hierarchy

```
UnionAll <: Type{T} <: Any
```


In [57]:
# Short for `broadcast(*, vector, rowvector)
vector .* rowvector

3×3 Array{Int64,2}:
 1  2  3
 2  4  6
 3  6  9

## Julia v1.0

Julia is currently at version v0.6. It's the 6th fully-functional iteration. The next version will be v1.0, released first half of next year.

v1.0 does not mean it is "finished" - just that the syntax, semantics and API has become more stable. New features like a fully static compilation mode will follow after this, probably sometime during a series of non-breaking v1.x  releases.

## Conclusion

To conclude, what I enjoy about Julia is how it encourages clear, concise and generic code. 

 * Clear - code is easy to read, a functional programming style is encouraged, a lot of effort to make sensible APIs
 * Concise - syntax, standard library and packages mean I can acheive a lot with little effort
 * Generic - functions I write can work with a variety of types, including some I don't know about.
 
The last point is *super* important. Power comes from combining orthogonal features together! 

###  Some interesting packages

The pakage ecosystem and our own packages provide useful functionality for dealing with everyday numerical tasks at Roames. 

Some we own or contribute to:

 * *StaticArrays* - fast, stack-allocated, SIMD arrays for e.g. 3D geometry
 * *Rotations* - a variety of 3D rotation representations (3x3 matrices, quaternions, etc)
 * *CoordinateTransformations* - a system for composing and inverting geometric transformations
 * *Geodesy* - extends CoordinateTransformations to provide e.g. lat-lon to UTM transformations with various datums.
 * *Chrono* - scientific timekeeping
 * *FunctionalPointClouds* - spatially indexed point cloud data. Works a bit like a table or dataframe.
 * *RoamesPointClassifiers* - uses point cloud spatial features + XGBoost to classify points
 
To explore the package ecosystem, I recommend [juliaobserver.com](https://juliaobserver.com). Some nice packages for starting with from the ecosystem

 * *Plots* - a plotting framework with many plotting backends
 * *DataFrames* - tabular data like Python's *pandas* or R's *data.frame*
 * *XGBoost* - wrapper to XGBoost C library
 * *TensorFlow* - wrapper for tensorflow
 * *Flux* and *Mocha* - native Julia deep learning libraries
 * *DifferentialEquations* - extremely flexible D.E. solver framework (best in class?)
 * *JuMP*, *Convex* - mathematical programming library (solving optimization problems)
 * *Cxx* - Julia interface to (templated, uninstatiated) C++ code and JIT C++ compiler using clang/LLVM.
 